In [2]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_setup_function
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_setup_function)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_setup_function import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *

#### Lấy dữ liệu Mongo

In [3]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [4]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "pct_change": 1,
    "diff": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_prev_high": 1,
    "week_prev_low": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "year_open": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
	"YFIBO_0382": 1,
    "YFIBO_0500": 1,
    "YFIBO_0618": 1,
    'WPIVOT_P': 1,
    'MPIVOT_P': 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [5]:
itd_index_df = get_mongo_collection(stock_db, "itd_index")
other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

#### Tạo các models cho gemini

In [6]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

#### Dữ liệu thống kê giao dịch phiên hôm trước

- Tính toán bảng dữ liệu

In [7]:
itd_vn30f1m_df = itd_index_df[itd_index_df['ticker'] == 'VN30F1M'].reset_index(drop=True)
eod_vn30f1m_df = full_index_df[full_index_df['ticker'] == 'VN30F1M'].reset_index(drop=True)
eod_vn30_df = full_index_df[full_index_df['ticker'] == 'VN30'].reset_index(drop=True)
vn30f1m_close_prev = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['close'].item()
vn30_close_prev = eod_vn30_df[eod_vn30_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['close'].item()

itd_vn30f1m_df['cum_volume'] = itd_vn30f1m_df['volume'][::-1].cumsum()[::-1]
itd_vn30f1m_df['pct_change'] = (itd_vn30f1m_df['close'] - vn30f1m_close_prev) / vn30f1m_close_prev
itd_vn30f1m_df['diff'] = itd_vn30f1m_df['close'] - vn30f1m_close_prev
itd_vn30f1m_df['basis'] = itd_vn30f1m_df['close'] - vn30_close_prev
itd_vn30f1m_df['open_interest'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['option'].item()

itd_vn30f1m_df['week_open'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['week_open'].item()
itd_vn30f1m_df['week_prev_high'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['week_prev_high'].item()
itd_vn30f1m_df['week_prev_low'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['week_prev_low'].item()
itd_vn30f1m_df['week_pivot'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['WPIVOT_P'].item()

itd_vn30f1m_df['month_open'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['month_open'].item()
itd_vn30f1m_df['month_prev_high'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['month_prev_high'].item()
itd_vn30f1m_df['month_prev_low'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['month_prev_low'].item()
itd_vn30f1m_df['month_pivot'] = eod_vn30f1m_df[eod_vn30f1m_df['date'] < itd_vn30f1m_df['date'].iloc[-1].normalize()].iloc[0]['MPIVOT_P'].item()

daily_8h45_itd_df = itd_vn30f1m_df.sort_values('date').copy()

- Viêt nhận xét ngắn

In [8]:
def daily_vn30f1m_overall_comment(df):
    prompt = f"""
    Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M:
    {df.to_csv(index=False, sep='|', lineterminator='\n')}

    **Bối cảnh:**
    Hãy đóng vai là một nhà phân tích tài chính chuyên nghiệp và khách quan của một công ty chứng khoán. Nhiệm vụ của bạn là phân tích dữ liệu giá theo phút của hợp đồng tương lai (ticker VN30F1M) được cung cấp và viết một báo cáo ngắn gọn, súc tích về diễn biến của phiên giao dịch đó.

    **Dữ liệu đầu vào:**
    Dữ liệu là một chuỗi văn bản chứa thông tin giao dịch theo từng phút, bao gồm các cột: `date`, `ticker`, `open`, `high`, `low`, `close`.

    **Yêu cầu đầu ra:**
    Kết quả trả về phải tuân thủ nghiêm ngặt các quy tắc sau:

    * **Định dạng:** Chỉ trả về **duy nhất** đoạn văn phân tích. **Tuyệt đối không** thêm bất kỳ câu dẫn dắt, tiêu đề, hay lời chào nào (ví dụ: không được viết "Dưới đây là báo cáo..." hoặc "Sau khi phân tích...").
    * **Cấu trúc:** Đoạn văn phải có chính xác **3 câu**. Mỗi câu từ 25 đến 30 từ.
    * **Giọng điệu:** Chuyên nghiệp, khách quan, mang tính tường thuật, không đưa ra khuyến nghị mua/bán.
    * **Nguồn:** Mọi phân tích phải dựa hoàn toàn vào dữ liệu được cung cấp.
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.

    **Chi tiết hướng dẫn cho từng câu:**

    * **Câu 1: Bối cảnh và Diễn biến mở đầu (khoảng 15-20 từ)**
        * **Mục đích:** Giới thiệu bối cảnh phiên giao dịch và mô tả xu hướng ngay sau khi thị trường mở cửa.
        * **Hướng dẫn nội dung:**
            1.  Bắt đầu câu bằng cụm từ "Trong phiên giao dịch ngày `[DD/MM/YYYY]`...". Bạn phải tự động trích xuất và định dạng ngày từ cột `date` trong dữ liệu.
            2.  Nhận xét ngắn gọn về trạng thái mở cửa (ví dụ: "mở cửa khá ổn định", "mở cửa trong sắc xanh", "chịu áp lực ngay từ đầu").
            3.  Mô tả hành động giá ngay sau đó (ví dụ: "nhưng nhanh chóng chịu áp lực bán", "tuy nhiên đà tăng không duy trì được", "và tiếp tục nới rộng đà giảm").

    * **Câu 2: Diễn biến chính và Mức biến động (khoảng 15-20 từ)**
        * **Mục đích:** Tóm tắt xu hướng chủ đạo trong phiên và định lượng mức độ biến động mạnh nhất.
        * **Hướng dẫn nội dung:**
            1.  Mô tả xu hướng chính kéo dài trong phiên (ví dụ: "Xu hướng tiêu cực này chiếm ưu thế", "Lực mua chủ động xuyên suốt phiên").
            2.  Làm nổi bật mức biến động lớn nhất bằng cách tính chênh lệch giữa giá trị `high` cao nhất và `low` thấp nhất trong toàn bộ dữ liệu. Diễn đạt nó dưới dạng "đẩy chỉ số có thời điểm giảm hơn `X` điểm so với mức cao nhất" hoặc "giúp chỉ số có lúc tăng hơn `X` điểm so với mức điểm thấp nhất" .

    * **Câu 3: Tổng kết cuối phiên (khoảng 15-20 từ)**
        * **Mục đích:** Mô tả các nỗ lực vào cuối phiên và chốt lại trạng thái của giá đóng cửa.
        * **Hướng dẫn nội dung:**
            1.  Nhận xét về các diễn biến trong giai đoạn cuối phiên (ví dụ: "Mặc dù một nỗ lực phục hồi...", "Áp lực bán gia tăng trở lại...", "Xu hướng vẫn tiếp diễn...").
            2.  Đưa ra kết luận về giá đóng cửa so với mặt bằng chung của phiên (ví dụ: "chỉ số vẫn duy trì ở vùng giá thấp", "bảo toàn được thành quả", "thu hẹp đáng kể đà giảm").
    """
    return prompt

overall_vn30f1m_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_overall_comment(itd_vn30f1m_df), 'daily_vn30f1m_overall_comment')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_overall_comment.


#### Lấy dữ liệu nntd VN30F1M

In [9]:
# Đọc tất cả các file FiinProX từ thư mục Downloads
fiinprox_files = glob.glob(os.path.join(str(Path.home() / "Downloads"), "*FiinProX*.xlsx"))

# Lọc lại các file chứa dữ liệu thị trường mở và có số cột > 6
td_finprox_files = []
for file in fiinprox_files:
    df = pd.read_excel(file)
    td_name_check = df.applymap(lambda x: isinstance(x, str) and "Thống kê thị trường_Tự doanh-Chi tiết" in x).any().any()
    if td_name_check:
        td_finprox_files.append(file)
if len(td_finprox_files) == 0:
    raise FileNotFoundError("❌ Chưa tải dữ liệu Tự Doanh từ FiinProX.")

# Sắp xếp fiinprox_files theo thời gian tạo (tăng dần) và lấy file mới nhất
last_td_fiinprox_file = sorted(td_finprox_files, key=os.path.getctime)[-1]

In [10]:
#Đọc dữ liệu Tự Doanh từ file mới nhất
td_vn30f1m_df = pd.read_excel(last_td_fiinprox_file, skiprows=8).iloc[:, :12]
td_vn30f1m_df = td_vn30f1m_df.dropna().iloc[:20].reset_index(drop=True)
td_vn30f1m_df = td_vn30f1m_df.drop(td_vn30f1m_df.columns[[3, 5, 7, 9]], axis=1)
td_vn30f1m_df.columns = ['ticker', 'date', 'buy_volume', 'sell_volume', 'buy_value', 'sell_value', 'net_value', 'net_volume']
td_vn30f1m_df['sell_volume'] = -td_vn30f1m_df['sell_volume']
td_vn30f1m_df['sell_value'] = -td_vn30f1m_df['sell_value']
td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]] = td_vn30f1m_df[[col for col in td_vn30f1m_df.columns if '_value' in col]]/1000
td_vn30f1m_df['type'] = 'TD'

#Kiểm tra xem dữ liệu Tự Doanh đã mới nhất chưa
td_date_check = td_vn30f1m_df['date'].max()
date_to_check = date_series.iloc[2].item()
if td_date_check < date_to_check:
    raise ValueError(f"❌ Dữ liệu Tự Doanh không mới nhất. Ngày trong dữ liệu: {td_date_check}, ngày yêu cầu: {date_to_check}")
else:
    print(f"✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: {td_date_check.strftime('%d-%m-%Y')}")

#Lưu riêng ticker cho VN30F1M dùng sau này
vn30f1m_ticker = td_vn30f1m_df['ticker'].unique()[0]

#Dữ liệu NN từ Fireant
nn_vn30f1m_df = nntd_index_df[(nntd_index_df['ticker'] == 'VN30F1M') & (nntd_index_df['type'] == 'NN')].reset_index(drop=True)
nn_vn30f1m_df['ticker'] = vn30f1m_ticker

#Ghép bảng NN và TD
daily_8h45_nntd_df = pd.concat([nn_vn30f1m_df, td_vn30f1m_df], axis=0, ignore_index=True)

✅ Đã cập nhật dữ liệu Tự Doanh đã mới nhất. Ngày trong dữ liệu: 14-08-2025


#### Vẽ biểu đồ kĩ thuật

In [ ]:
vn30f1m_chart_df = full_index_df[full_index_df['ticker'] == 'VN30F1M'].iloc[:120].sort_values('date').reset_index(drop=True)
vn30f1m_chart_df['ticker'] = vn30f1m_ticker
image_name = 'TA_DAILY_VN30F1M.png'

line_name_dict = {
    'SMA_20': 'SMA 20',
    'SMA_60': 'SMA 60',
    'week_open': 'WEEK OPEN',
    'month_open': 'MONTH OPEN',
    'quarter_open': 'QUARTER OPEN',
    'year_open': 'YEAR OPEN',
    'month_prev_high': 'LAST MHIGH',
    'month_prev_low': 'LAST MLOW',
    'quarter_prev_high': 'LAST QHIGH',
    'quarter_prev_low': 'LAST QLOW',
    'MFIBO_0382': 'MFIBO 0.382',
    'MFIBO_0500': 'MFIBO 0.500',
    'MFIBO_0618': 'MFIBO 0.618',
    'QFIBO_0382': 'QFIBO 0.382',
    'QFIBO_0500': 'QFIBO 0.500',
    'QFIBO_0618': 'QFIBO 0.618',
	'YFIBO_0382': 'YFIBO 0.382',
    'YFIBO_0500': 'YFIBO 0.500',
    'YFIBO_0618': 'YFIBO 0.618'
}

chart_config = create_chart_config(
	title_font_size=30,
	axis_font_size=24,
	tag_font_size=24,
	price_tag_font_size=24,
    min_spacing_ratio = 0.042,
	margin=dict(l=20, r=220, t=20, b=20, pad=10)
)

max_attempts = 5
for attempt in range(max_attempts):
    try:
        fig, fig_images = create_financial_chart(
            vn30f1m_chart_df, # DataFrame của bạn
            width=1400,
            height=1200,
            line_name_dict=line_name_dict,
            line_columns=list(line_name_dict.keys()),
            chart_config=chart_config,
            path='../output/daily/',
            symbol_name='VN30F1M',
            image_name = image_name
        )
        upload_to_r2(fig_images, image_name)
        break
    except Exception as e:
        if attempt == max_attempts - 1:
            raise e
# fig.show()

In [14]:
def daily_vn30f1m_ta_analysis(itd_df, eod_df):
    prompt = f"""
Đây là dữ liệu trong phiên gần nhất của chỉ số hợp đồng tương lai VN30F1M:
    {itd_df[['date', 'ticker', 'open', 'high', 'low', 'close']].to_csv(index=False, sep='|', lineterminator='\\n')}

Đây là dữ liệu lịch sử của chỉ số hợp đồng tương lai VN30F1M:
    {eod_df.to_csv(index=False, sep='|', lineterminator='\\n')}

**Vai trò:** Bạn là một **nhà phân tích kỹ thuật cấp cao**, chuyên về phái sinh, có nhiệm vụ đưa ra nhận định kỹ thuật khách quan và sắc bén.

**Nhiệm vụ:** Dựa trên dữ liệu và các chỉ báo kỹ thuật, hãy viết một báo cáo phân tích về trạng thái kỹ thuật của chỉ số cho **phiên kế tiếp**, tuân thủ nghiêm ngặt cấu trúc **8 câu, 3 đoạn**.

**Yêu cầu về nội dung, cấu trúc và định dạng:**
    Báo cáo phải gồm **8 câu, mỗi câu dài khoảng 20 đến 25 từ** và được chia thành **3 đoạn văn riêng biệt**. Bắt đầu thẳng vào nội dung, không có lời chào.

    **Đoạn 1 (3 câu): Phân tích Phiên Gần Nhất dựa vào dự liệu trong phiên gần nhất**
        * **Câu 1:** Viết một câu duy nhất mô tả phiên gần nhất theo đúng cấu trúc sau: **"Trong phiên (dd/mm), chỉ số [Trạng thái mở cửa so với phiên trước] và [Diễn biến ngắn gọn đầu và cuối phiên]"** đã hình thành nên một **"[Mô tả ngắn gọn về cây nến (nến tăng, nến giảm hoặc nến lưỡng lự), sử dụng giá OHLC ngày cuối cùng]"**, thể hiện **"[Kết luận về phe thắng thế hoặc sự giằng co trong phiên]"**.
        * **Câu 2:** Phân tích vai trò của hai đường **SMA 20 và SMA 60** trong việc xác nhận xu hướng ngắn hạn và trung hạn hiện tại.
        * **Câu 3:** Đánh giá chỉ báo **RSI 14**, nêu bật trạng thái hiện tại của chỉ báo (ví dụ: vùng trung tính, quá mua/bán) và ý nghĩa của nó.

    **Đoạn 2 (3 câu): Xác định các Vùng giá Cốt lõi dựa vào giữ liệu lịch sử**
        * **Câu 4:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tuần** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 5:** Phân tích và xác định mốc hỗ trợ/kháng cự quan trọng theo **khung Tháng** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        * **Câu 6:** Đưa ra kết luận và nhấn mạnh **một vùng hỗ trợ và một vùng kháng cự then chốt nhất** sẽ quyết định xu hướng trong các phiên tới.

    **Đoạn 3 (2 câu): Kết luận Kỹ thuật**
        * **Câu 7:** Đưa ra một câu nhận định tổng thể, khẳng định xu hướng kỹ thuật chủ đạo hiện tại của chỉ số (tăng, giảm hoặc đi ngang).
        * **Câu 8:** Ngay sau đó, chỉ ra các vùng giá hỗ trợ và kháng cự then chốt sẽ là tâm điểm quyết định cho các phiên giao dịch tiếp theo và giải thích tại sao.

**Yêu cầu về ngôn ngữ và trình bày:**
    * **Văn phong:** Sắc bén, khách quan, quyết đoán và đa dạng, tránh lặp lại cấu trúc câu.
    * **TUYỆT ĐỐI KHÔNG** sử dụng dấu ;.
    * **TUYỆT ĐỐI KHÔNG** được viết tên cột dữ liệu gốc và **KHÔNG** có lời chào hay câu dẫn dắt.
    * **TUYỆT ĐỐI KHÔNG** sử dụng các từ nối mang tính tổng kết ở đầu câu, cứ đi thẳng vào nội dung chính (ví dụ: "Tóm lại", "Nhìn chung", "Do đó", "Theo đó", "Tổng thể").
    * **LUÔN LUÔN** sử dụng ngày dạng "dd/mm" trong bài viết khi được yêu cầu.
    """
    return prompt

ta_vn30f1m_comment = generate_content_with_model_dict(standard_model_dict, daily_vn30f1m_ta_analysis(itd_vn30f1m_df, vn30f1m_chart_df), 'daily_vn30f1m_ta_analysis')


✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_ta_analysis.


#### Chiến lược mua bán

In [18]:
def prompt_vn30f1m_long(df, ta_vn30f1m_comment):
    return f"""
Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M (OHLCV và chỉ báo):
{df.to_csv(index=False, sep='|', lineterminator='\\n')}

Đây là nhận định từ chuyên gia về xu hướng thị trường:
{ta_vn30f1m_comment}

**Bối cảnh và Vai trò**
Bạn là một nhà phân tích kỹ thuật cao cấp tại quỹ đầu tư, chuyển hóa dữ liệu và nhận định thành kế hoạch hành động có thể thực thi ngay trong phiên kế tiếp. Giá hiện tại = giá Close của bản ghi cuối cùng.

**Yêu cầu đầu ra — Cấu trúc và Nội dung**
Đầu ra phải gồm ĐÚNG 3 đoạn văn, mỗi đoạn ĐÚNG 2 câu, và mỗi câu kết thúc bằng dấu chấm (.). Tuyệt đối không sử dụng dấu chấm phẩy (;) hoặc bất kỳ ký tự phân cách câu nào khác. Không có tiêu đề, không bullet, không câu dẫn, và không thêm bất kỳ câu hay đoạn văn nào khác ngoài 3 đoạn yêu cầu. Văn phong chuyên nghiệp, súc tích, khách quan, và có tính liền mạch. 
**Độ dài mỗi câu văn chính xác trong khoảng từ 20 tới 25 từ.**

**Nội dung chi tiết cho từng đoạn:**

**Đoạn 1: Điểm vào lệnh Long tiềm năng và lý do**
- Trong hai câu, hãy trình bày hai vùng vào lệnh Long tiềm năng nhất, đồng thời nêu rõ lý do kỹ thuật cụ thể cho từng điểm vào.

**Đoạn 2: Mục tiêu chốt lời tiềm năng và tiêu chí xác nhận**
- Trong hai câu, hãy đề xuất hai mục tiêu Chốt Lời cho vị thế Long, xác định rõ mốc giá hoặc vùng cụ thể cùng với tiêu chí kỹ thuật cần đạt để xác nhận thành công mỗi mục tiêu.

**Đoạn 3: Vùng cắt lỗ và tiêu chí xác nhận**
- Trong hai câu, hãy xác định các vùng Cắt Lỗ cụ thể (cách các điểm vào lệnh đã trình bày ở trên bám sát mốc kỹ thuật quan trọng, ưu tiên mốc kĩ thuật gần nhất, nếu không có mốc nào gần thị gọn vùng tròn số cách khoảng 5 điểm), đồng thời nêu rõ tiêu chí kỹ thuật cần đạt để xác nhận việc cắt lỗ.

- *Lưu ý quan trọng*: Chỉ dùng TIẾNG VIỆT và Tuyệt đối không dùng từ "mua" trong toàn bộ đầu ra, nên ưu tiên dùng "vị thế Long" hoặc các cụm từ mang hàm ý tương tự.
    """

long_comment = generate_content_with_model_dict(standard_model_dict, prompt_vn30f1m_long(vn30f1m_chart_df, ta_vn30f1m_comment), 'daily_vn30f1m_long')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_long.


In [ ]:
def prompt_vn30f1m_short(df, ta_vn30f1m_comment):
    return f"""
Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M (OHLCV và chỉ báo):
{df.to_csv(index=False, sep='|', lineterminator='\\n')}

Đây là nhận định từ chuyên gia về xu hướng thị trường:
{ta_vn30f1m_comment}

**Bối cảnh và Vai trò**
Bạn là một nhà phân tích kỹ thuật cao cấp tại quỹ đầu tư, chuyển hóa dữ liệu và nhận định thành kế hoạch hành động có thể thực thi ngay trong phiên kế tiếp. Giá hiện tại = giá Close của bản ghi cuối cùng.

**Yêu cầu đầu ra — Cấu trúc và Nội dung**
Đầu ra phải gồm ĐÚNG 3 đoạn văn, mỗi đoạn ĐÚNG 2 câu, và mỗi câu kết thúc bằng dấu chấm (.). Tuyệt đối không sử dụng dấu chấm phẩy (;) hoặc bất kỳ ký tự phân cách câu nào khác. Không có tiêu đề, không bullet, không câu dẫn, và không thêm bất kỳ câu hay đoạn văn nào khác ngoài 3 đoạn yêu cầu. Văn phong chuyên nghiệp, súc tích, khách quan, và có tính liền mạch. 
**Độ dài mỗi câu văn chính xác trong khoảng từ 20 tới 25 từ.**

**Nội dung chi tiết cho từng đoạn:**

**Đoạn 1: Điểm vào lệnh Short tiềm năng và lý do**
- Trong hai câu, hãy trình bày hai vùng vào lệnh Short tiềm năng nhất, đồng thời nêu rõ lý do kỹ thuật cụ thể cho từng điểm vào.

**Đoạn 2: Mục tiêu chốt lời tiềm năng và tiêu chí xác nhận**
- Trong hai câu, hãy đề xuất hai mục tiêu Chốt Lời cho vị thế Short, xác định rõ mốc giá hoặc vùng cụ thể cùng với tiêu chí kỹ thuật cần đạt để xác nhận thành công mỗi mục tiêu.

**Đoạn 3: Vùng cắt lỗ và tiêu chí xác nhận**
- Trong hai câu, hãy xác định các vùng Cắt Lỗ cụ thể (cách các điểm vào lệnh đã trình bày ở trên bám sát mốc kỹ thuật quan trọng, ưu tiên mốc kĩ thuật gần nhất, nếu không có mốc nào gần thì gọn vùng tròn số cách khoảng 5 điểm), đồng thời nêu rõ tiêu chí kỹ thuật cần đạt để xác nhận việc cắt lỗ.

- *Lưu ý quan trọng*: Chỉ dùng TIẾNG VIỆT Tuyệt đối không dùng từ "bán" trong toàn bộ đầu ra, nên ưu tiên dùng "vị thế Short" hoặc các cụm từ mang hàm ý tương tự.
    """

short_comment = generate_content_with_model_dict(standard_model_dict, prompt_vn30f1m_short(vn30f1m_chart_df, ta_vn30f1m_comment), 'daily_vn30f1m_short')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_short.


In [14]:
def prompt_vn30f1m_risk(long_comment, short_comment):
    return f"""
Đây là nhận định từ chuyên gia về vị thế Long:
{long_comment}

Đây là nhận định từ chuyên gia về vị thế Short:
{short_comment}

**Bối cảnh và Vai trò**
Bạn là một nhà phân tích kỹ thuật cao cấp tại quỹ đầu tư, chuyển hóa dữ liệu và nhận định thành kế hoạch hành động có thể thực thi ngay trong phiên kế tiếp. Giá hiện tại = giá Close của bản ghi cuối cùng.

**Yêu cầu đầu ra — Cấu trúc và Nội dung**
Đầu ra phải gồm ĐÚNG 1 đoạn văn, trong đó có ĐÚNG 2 câu, và mỗi câu kết thúc bằng dấu chấm (.). Tuyệt đối không sử dụng dấu chấm phẩy (;) hoặc bất kỳ ký tự phân cách câu nào khác. Không có tiêu đề, không bullet, không câu dẫn, và không thêm bất kỳ câu hay đoạn văn nào khác ngoài 3 đoạn yêu cầu. Văn phong chuyên nghiệp, súc tích, khách quan, và có tính liền mạch. 
**Độ dài mỗi câu văn chính xác trong khoảng từ 20 tới 30 từ.**

**Hướng dẫn chi tiết cho từng câu**
- Từ các nhận xét về vị thế Long và Short, hãy trình bày các quy tắc quản trị vốn tổng quát (không trình bày riêng lẻ từng vị thế) bao gồm tỷ lệ rủi ro cho mỗi lệnh và tổng rủi ro, cùng với điều kiện giảm quy mô hoặc thoát một phần khi có biến động bất lợi.
"""

risk_comment = generate_content_with_model_dict(standard_model_dict, prompt_vn30f1m_risk(long_comment, short_comment), 'daily_vn30f1m_risk')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_risk.


In [15]:
def prompt_vn30f1m_evaluation(df, ta_vn30f1m_comment):
    return f"""
Đây là dữ liệu của chỉ số hợp đồng tương lai VN30F1M (OHLCV và chỉ báo):
{df.to_csv(index=False, sep='|', lineterminator='\\n')}

Đây là nhận định từ chuyên gia về xu hướng thị trường:
{ta_vn30f1m_comment}

**Bối cảnh và Vai trò**
Bạn là một nhà phân tích kỹ thuật cao cấp tại quỹ đầu tư, chuyển hóa dữ liệu và nhận định thành kế hoạch hành động có thể thực thi ngay trong phiên kế tiếp. Giá hiện tại = giá Close của bản ghi cuối cùng.

**Yêu cầu đầu ra**

- Viết ĐÚNG 1 đoạn văn duy nhất, trong đó có ĐÚNG 3 câu, **độ dài mỗi câu văn chính xác trong khoảng từ 20 tới 25 từ.**, bắt đầu trực tiếp, không có câu giới thiệu.
- Mỗi câu kết thúc bằng dấu chấm (.). Tuyệt đối không sử dụng dấu chấm phẩy (;) hoặc bất kỳ ký tự phân cách câu nào khác. Không có tiêu đề, không bullet, không câu dẫn, và không thêm bất kỳ câu hay đoạn văn nào khác ngoài 3 đoạn yêu cầu. Văn phong chuyên nghiệp, súc tích, khách quan, và có tính liền mạch. 

** Hướng dẫn chi tiết cho từng câu:**
- Câu 1: Kết luận nên ưu tiên Vị thế Long hay Vị thế Short trong giai đoạn này.
- Câu 2: Giải thích ngắn gọn dựa trên trạng thái của giá so với các mốc kỹ thuật chủ đạo và xu hướng hiện tại trong dữ liệu/nhận định.
- Câu 3: Nêu các lưu ý quan trọng nếu chọn chiến lược còn lại, gồm điều kiện vô hiệu, vùng cần xác nhận, và khuyến nghị quản trị rủi ro.
"""

evaluation_comment = generate_content_with_model_dict(standard_model_dict, prompt_vn30f1m_evaluation(vn30f1m_chart_df, ta_vn30f1m_comment), 'daily_vn30f1m_evaluation')

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm daily_vn30f1m_evaluation.


#### Lưu vào MSSQL

In [22]:
daily_8h45_comment_df = pd.DataFrame(
    {
        "type":     ["overall", "ta_index", "long", "short", "risk", "evaluation", "date", "time"],
        "comment": [
            overall_vn30f1m_comment,
            ta_vn30f1m_comment,
            long_comment,
            short_comment,
            risk_comment,
            evaluation_comment,
            time_series.iloc[0].item().strftime('%d/%m/%Y'),
            time_series.iloc[0].item().strftime('%H:%M')
        ],
    }
)


In [23]:
%%capture
save_to_mssql(cts_engine, daily_8h45_comment_df, 'daily_8h45_comment')
save_to_mssql(cts_engine, daily_8h45_itd_df, 'daily_8h45_itd')
save_to_mssql(cts_engine, daily_8h45_nntd_df, 'daily_8h45_nntd')